In [1]:
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_google_genai import GoogleGenerativeAI
from datetime import date
from langchain.agents import tool
import langchain
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

c:\Users\sharu\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from langchain.llms.bedrock import Bedrock
# import boto3

In [5]:
langchain.debug=True

In [6]:
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 64,
  "response_mime_type": "text/plain",
}

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ['GOOGLE_API_KEY'],generation_config=generation_config)

In [8]:
# client = boto3.client('bedrock-runtime', region_name='us-east-1')

# llm=Bedrock(client=client,
#             model_id="anthropic.claude-instant-v1",
#             model_kwargs={
#                 "max_tokens_to_sample":500,
#                 "temperature":0,
#                 "top_p":0.9
#             }
#         )

In [9]:
# @tool
# def time(text: str) -> str:
#     """Returns todays date, use this for any \
#     questions related to knowing todays date. \
#     The input should always be an empty string, \
#     and this function will always return todays \
#     date."""
#     # return str(date.today())
#     return str("hai")

In [10]:
vectordb=None

In [11]:
from flask import Flask, render_template_string
import threading
app = Flask(__name__)
thread=None

@tool
def host_webpage(html_content):
    """
    Returns the link where question webpage hosted, use this after \
    questions are generated in html form format. \
    The input should always be an html form code, \
    and this function will always return link to webpage hosted  \
    """

    @app.route('/')
    def display_html():
        return render_template_string(html_content)

    def run_app():
        app.run(host='localhost', port=5000, debug=False)

    # Start Flask in a separate thread
    global thread
    thread = threading.Thread(target=run_app)
    thread.start()

    # Return the link to the page
    return "http://localhost:5000/"

In [12]:
@tool
def get_revelent_context(topic: str) -> str:
    """Returns the relevent context to generate questions, use this after \
    the user have choosed the topic to generate questions. \
    The input should always be an topic that user mentioned, \
    and this function will always return context string  \
    """
    
    relevent_doc=vectordb.similarity_search(topic,k=5)
    context="\n\n".join([doc.page_content for doc in relevent_doc])

    return str(context)

In [13]:
@tool
def get_topics_from_uploaded_document(text: str) -> str:
    """Returns the topics in the uploaded documents, use this after \
    document loaded successfully. \
    The input should always be an empty string, \
    and this function will always return string of topics \
    ."""
    # return str(date.today())
    return str("aws workflows, aws redshift copy")

In [14]:
@tool
def upload_doc(text: str) -> str:
    """loads the document to memory. Use only when user mentioned i want to upload something\
       or else don't use.The input should always be an empty string. \
       and this function always returns status of the upload."""
    # return str(date.today())

    path=input("please enter path")
    print("Loading the data.....")
    loader = PyPDFLoader(path)
    data = loader.load()
    docs=text_splitter.split_documents(data)
    global vectordb 
    vectordb= Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        # persist_directory='docs/chroma/'
    )
    return str("uploaded successfully")

In [15]:
tools=[upload_doc,get_topics_from_uploaded_document,get_revelent_context,host_webpage]

In [16]:
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [17]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    conversational=True,
    max_iterations=5,
    verbose = True)

c:\Users\sharu\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
template=""" 
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

You are an user-friendly MCQ Generating Assistant. Answer the human message accordingly and generate multiple-choice questions (MCQs) as best you can.
always follow these steps:

1. Greet the user and ask for the source of the content for the questions (Upload a PDF document, Provide a URL to a web page, Enter text directly).
2. Based on the chosen source, take the appropriate action:
    - Upload: Call `upload_doc` with the user-uploaded document.
    - URL: Access the web page and extract content.
    - Text: Use the provided text directly.
3. Ask the user for the difficulty level of the questions (e.g., Easy, Medium, Hard).
4. Ask for the Topic and generate 5 Question on the selected topic.
5. Generate questions in a HTML form format.
6. Call `host_webpage` with the generated HTML code and provide the link to the user.

"""

In [21]:
agent.agent.llm_chain.prompt.messages[0].prompt.template=template

In [22]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

 
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of ta

In [28]:
chat_history=[]
response=agent({"input": "hi", "chat_history": chat_history})

c:\Users\sharu\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "hi",
  "chat_history": []
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "hi",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilitie

In [29]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [30]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?')]

In [31]:
response=agent({"input": "i want to upload a document", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [32]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)


In [33]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you.")]

In [34]:
response=agent({"input": "what are the topics available in document", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [35]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [36]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you."),
 HumanMessage(content='what are the topics available in document'),
 AIMessage(content='The topics available in the document are: aws workflows, aws redshift copy.')]

In [37]:
response=agent({"input": "Okay generate mcq questions on aws workflows", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [38]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [39]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you."),
 HumanMessage(content='what are the topics available in document'),
 AIMessage(content='The topics available in the document are: aws workflows, aws redshift copy.'),
 HumanMessage(content='Okay generate mcq questions on aws workflows'),
 AIMessage(content='Amazon Managed Workflows for Apache Airflow (MWAA) is a managed service that simplifies the use of Apache Airflow, a popular workflow orchestration tool. It eliminates the need for installing and maintaining Airflow, allowing you to focus on defining and managing your workflows. MWAA pro

In [40]:
response=agent({"input": "can you generate mcq's on the same", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [41]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [42]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you."),
 HumanMessage(content='what are the topics available in document'),
 AIMessage(content='The topics available in the document are: aws workflows, aws redshift copy.'),
 HumanMessage(content='Okay generate mcq questions on aws workflows'),
 AIMessage(content='Amazon Managed Workflows for Apache Airflow (MWAA) is a managed service that simplifies the use of Apache Airflow, a popular workflow orchestration tool. It eliminates the need for installing and maintaining Airflow, allowing you to focus on defining and managing your workflows. MWAA pro

In [43]:
response=agent({"input": "generate medium type mcq's", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [44]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [45]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you."),
 HumanMessage(content='what are the topics available in document'),
 AIMessage(content='The topics available in the document are: aws workflows, aws redshift copy.'),
 HumanMessage(content='Okay generate mcq questions on aws workflows'),
 AIMessage(content='Amazon Managed Workflows for Apache Airflow (MWAA) is a managed service that simplifies the use of Apache Airflow, a popular workflow orchestration tool. It eliminates the need for installing and maintaining Airflow, allowing you to focus on defining and managing your workflows. MWAA pro

In [46]:
response=agent({"input": "get revelent context and generate", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

In [47]:
chat_history.extend(
    [
        HumanMessage(content=response['input']),
        AIMessage(content=response["output"]),
    ]
)

In [48]:
chat_history

[HumanMessage(content='hi'),
 AIMessage(content='Hi! 👋  To get started, please tell me what you want to create MCQs from. Do you want to upload a document, provide a URL to a web page, or enter text directly?'),
 HumanMessage(content='i want to upload a document'),
 AIMessage(content="Great! Now, can you please tell me what topics are covered in the document you uploaded? I'll use this information to create some multiple-choice questions for you."),
 HumanMessage(content='what are the topics available in document'),
 AIMessage(content='The topics available in the document are: aws workflows, aws redshift copy.'),
 HumanMessage(content='Okay generate mcq questions on aws workflows'),
 AIMessage(content='Amazon Managed Workflows for Apache Airflow (MWAA) is a managed service that simplifies the use of Apache Airflow, a popular workflow orchestration tool. It eliminates the need for installing and maintaining Airflow, allowing you to focus on defining and managing your workflows. MWAA pro

In [49]:
response=agent({"input": "can you host the same on webpage", "chat_history": chat_history})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  \nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative resp

 * Running on http://localhost:5000
Press CTRL+C to quit


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:GoogleGenerativeAI] [1.85s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"action\": \"Final Answer\",\n    \"action_input\": \"I have hosted the MCQs on a webpage for you. You can access it at http://localhost:5000/.  Let me know if you have any other questions!\"\n}\n```",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:AgentExecutor > chain:LLMChain] [1.86s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"action\": \"Final Answer\",\n    \"action_input\": \"I have hosted the MCQs on a webpage for you. You can access it at http://localhost:5000/.  Let me know if you have any other questions!\"\n}\n```"
}
[chain/end] [chain:AgentExecutor] [7.78s] Exiting Chain run with output:
{
  "output": "I have hosted the MCQs on a webpage for you. You can access it at http://localhost:5000/

127.0.0.1 - - [18/Jul/2024 15:51:02] "GET / HTTP/1.1" 200 -
